In [ ]:
# Fra guiden i INF265
from matplotlib import pyplot as plt
import numpy as np
import torch
from torchvision import datasets, transforms
from collections import Counter
from torch.utils.data import random_split

torch.manual_seed(123)
